In [1]:
import datetime
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from chart import *
from table import *
from document import *

In [2]:
# 메타 정보

today = datetime.date.today()

y = today.year
m = today.month
d = today.day
w = today.weekday()

# report version
type = '월간'
month = '6월'
region = 'All' # All/KR|CN|NA|EU|SG|RU
v = '1'

if region == 'KR':
    regex = '^(KR|Global|All)'
elif region == 'CN':
    regex = '^(CN|Global|All)'
elif region == 'NA':
    regex = '^(NA|Global|All)'
elif region == 'EU':
    regex = '^(EU|Global|All)'
elif region == 'SG':
    regex = '^(SG|Global|All)'
elif region == 'RU':
    regex = '^(SG|Global|All)'
else:
    regex = '^(KR|CN|NA|EU|SG|RU|Global|All)'

print("보고서 유형: ", type, "월: ", month)
print(regex)

보고서 유형:  월간 월:  6월
^(KR|CN|NA|EU|SG|RU|Global|All)


In [3]:
# Document 기본 폰트
style = document.styles['Normal']
style.font.name = '맑은고딕'
style.font.size = Pt(12)
style._element.rPr.rFonts.set(qn('w:eastAsia'), '맑은고딕')

In [4]:
# Chart 기본 폰트
alt.themes.register('맑은고딕', hanfont)
alt.themes.enable('맑은고딕')

ThemeRegistry.enable('맑은고딕')

## 보고서 표지 (커버 쉬트)

In [5]:
if region == 'All':
    document.add_heading(f'[2022년.{month}] Cloud Infra 운영', level=0)
else:
    document.add_heading(f'[2022년.{month}] {region} Cloud Infra 운영', level=0)

document.add_heading(f'{y}.{m}.{d}', level=1)
document.add_heading(f'클라우드운영센터', level=1)
document.add_page_break()

## 요약

In [6]:
# ETL
base_path = r"./data/요약_202206_서비스관리_0727_01.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)

# 리전 & 운영계, 월간보고서용
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

In [7]:
print(len(df), len(df_filtered))

56 18


In [8]:
df_filtered[df_filtered['월'] == month]

,년도,월,리전,AZ,테넌트,진행상태,성패,작업내용,주간,월간
24,22년,6월,KR,2센터,PRD,완료,성공,F5 BIG-IP 취약점(CVE-2022-1388) 대응 펌웨어 업그레이드\n BI...,O,O
29,22년,6월,NA,1센터,PRD,완료,성공,CCS 2.0 RnR & 운영 업무 이관\n업무 인수인계 체크리스트\n(완료) 데이...,O,O
30,22년,6월,SG,1센터,PRD,완료,성공,F5 펌웨어 업그레이드\nF5 BIG-IP 제품군에 대한 iControl REST ...,O,O
50,22년,6월,EU,1센터,PRD,진행중,None,방화벽 IPS/FW 분리 및 IPSEC VPN 적용 요청 (→ 추가 요청 시 대응)...,O,O
52,22년,6월,EU,1센터,PRD,진행중,None,F5 펌웨어 업그레이드\nF5 BIG-IP 제품군에 대한 iControl REST ...,O,O
54,22년,6월,CN,1센터,PRD,진행중,None,클라우드 플랫폼(opentsack) 보안 설정 현황 확인 요청\n중국 GCS시스템에...,O,O
55,22년,6월,RU,1센터,PRD,진행중,None,러시아 CCS 운영계 Rescue노드 필요\n=> 러시아 CCS 운영계 Rescue...,O,O


In [9]:
# Data
pivot, total = getPivotTable(df, month)
source = pivot

# Chart
chart = getPieChart(source)

In [10]:
chart

alt.LayerChart(...)

In [11]:
# Chart
if region == 'All':
    # Data
    pivot, total = getPivotTable(df, month)
    source = pivot

    # Chart
    chart = getPieChart(source)
    chart.save(f'./charts/summary_{y}_{m}_{d}_{v}_1.png')

    # Docx
    document.add_paragraph('요약')    
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')
    document.add_picture(f'./charts/summary_{y}_{m}_{d}_{v}_1.png')
else:
    pass

npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.


In [12]:
# Document
addSummaryTable(df_filtered, regex, month)
document.add_page_break()

## 1.1 모니터링(인시던트핸들링)

In [13]:
base_path = r"./data/1_1_202206_모니터링_인시던트핸들링_0727_01.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df['장애전파소요시간'] = (pd.to_datetime(df['장애전파시간.1'])-pd.to_datetime(df['발생인지시간'])).astype('timedelta64[m]')

# Pivoting
pivot, total = getPivotTable(df, month)

# 리전 & 운영계, 월간보고서용
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료') & (df['월간']=='O')]
df_filtered, timeslot = getTables(df_filtered, regex, month)
    
# SLA failed
df_failed = df_filtered[ (df_filtered['장애전파'] > 10) & (df_filtered['월']== month)]
df_failed[['날짜', '조치내용', 'JIRA_Ticket_No', '장애전파소요시간']]

,날짜,조치내용,JIRA_Ticket_No,장애전파소요시간
92,2022.06.04,Deleted old snapshots,CNCLOUD2046,19.0
93,2022.06.08,Deleted old snapshots,None,30.0
100,2022.06.15,"Its known issue, it got resolved automatically",None,17.0
104,2022.06.02,오래된 스냅샷 삭제 후 알람 Resolved,None,2881.0
113,2022.06.09,Deleted old snapshots,None,211.0
132,2022.06.06,After restarting the Mangodb service on ccskro...,None,27.0
135,2022.06.06,Created the Ticket -> KRCLOUD-4994,KRCLOUD-4994,49.0
141,2022.06.11,Informed through kakao and owner said to ignor...,None,15.0
161,2022.06.30,21:12에 /postgres_data full 발생하여 DB down 발생하였습니...,KRCLOUD-5136,11.0
179,2022.06.21,"오창민 매니저 확인, 담당자 유선 및 메신저 컨택하여 확인해보니 서비스 배포작업 중...",None,11.0


In [14]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(source)
    chart.save(f'./charts/mon_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 스캐터 차트    
chart = getScatterChart(timeslot, month)
chart.save(f'./charts/mon_{region}_{y}_{m}_{d}_2.png')

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/mon_{region}_{y}_{m}_{d}_3.png')

In [15]:
# Document
document.add_paragraph('모니터링(인시던트핸들링)', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/mon_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'장애전파 소요시간(분)', style='List Bullet')
document.add_picture(f'./charts/mon_{region}_{y}_{m}_{d}_2.png')
## 평균 소요 시간, 최대 소요 시간

document.add_paragraph(f'장애전파시간 초과 이벤트', style='List Bullet')
addFailedTable(df_failed, regex, month)

document.add_paragraph(f'월별 발생 건수 (추세)', style='List Bullet')
document.add_picture(f'./charts/mon_{region}_{y}_{m}_{d}_3.png')   

In [16]:
# Document
document.add_page_break()

## 1.2 이슈 관리

In [17]:
base_path = r"./data/1_2_202206_이슈관리_0727_01.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df, month)

In [18]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(source)
    chart.save(f'./charts/im_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/im_{region}_{y}_{m}_{d}_2.png')

In [19]:
# Document
document.add_paragraph('이슈 관리', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/im_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 발생 건수 (추세)', style='List Bullet')
document.add_picture(f'./charts/im_{region}_{y}_{m}_{d}_2.png')   

In [20]:
# Document
addTable4(df_filtered, regex, month)
document.add_page_break()

## 1.3 장애 관리

In [21]:
base_path = r"./data/1_3_202206_장애관리_0727_01.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df, month)

In [22]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(source)
    chart.save(f'./charts/pm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/pm_{region}_{y}_{m}_{d}_2.png')

In [23]:
# Document
document.add_paragraph('장애(RCA) 관리', style='List Number')

if region == 'All':
    document.add_paragraph(f'6월: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/pm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 발생 건수 (추세)', style='List Bullet')
document.add_picture(f'./charts/pm_{region}_{y}_{m}_{d}_2.png')   

In [24]:
# Document
addTable4(df_filtered, regex, month)
document.add_page_break()

## 1.4 변경 관리

In [25]:
base_path = r"./data/1_4_202206_변경관리_0727_01.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df, month)

In [26]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(source)
    chart.save(f'./charts/cm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/cm_{region}_{y}_{m}_{d}_2.png')

In [27]:
# Document
document.add_paragraph('변경 관리', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/cm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 발생 건수 (추세)', style='List Bullet')
document.add_picture(f'./charts/cm_{region}_{y}_{m}_{d}_2.png')   

In [28]:
# Document
addTable3(df_filtered, regex, month)
document.add_page_break()

## 1.5 요청 관리

In [29]:
base_path = r"./data/1_5_202206_요청관리_0727_01.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df, month)

In [30]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(source)
    chart.save(f'./charts/cm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/cm_{region}_{y}_{m}_{d}_2.png')

In [31]:
# Document
document.add_paragraph('요청 관리', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/cm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 발생 건수 (추세)', style='List Bullet')
document.add_picture(f'./charts/cm_{region}_{y}_{m}_{d}_2.png')   

In [32]:
# Document
# addTable3(df_filtered, regex, month)
document.add_page_break()

## 1.6 자산 관리 - VM

In [33]:
base_path = r"./data/1_6_202206_자산관리_VM_0727_01.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df, month)

In [34]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(source)
    chart.save(f'./charts/vm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart2(df_filtered, month)
chart.save(f'./charts/vm_{region}_{y}_{m}_{d}_2.png')

In [35]:
# Document
document.add_paragraph('자산 관리(vm)', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/vm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 VM 수 (추세)', style='List Bullet')
document.add_picture(f'./charts/vm_{region}_{y}_{m}_{d}_2.png')   

In [36]:
# Document
# addSummaryTable(df_filtered, regex, month)
document.add_page_break()

## 1.6 자산 관리 - DB

In [37]:
base_path = r"./data/1_6_202206_자산관리_DB_0727_01.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df_filtered, month)
pivot2, total = getPivotTable2(df_filtered, month)
pivot2 = pivot2.reset_index()

In [38]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(source)
    chart.save(f'./charts/db_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# source = pivot2
# chart = getPieChart2(source)
# chart.save(f'./charts/db_{region}_{y}_{m}_{d}_2.png')

# Chart - 라인 차트
chart = getLineChart2(df_filtered, month)
chart.save(f'./charts/db_{region}_{y}_{m}_{d}_3.png')

In [39]:
# Document
document.add_paragraph('자산 관리(db)', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/db_{region}_{y}_{m}_{d}_1.png')
else:
    pass

addPivot2Table(pivot2)
# document.add_picture(f'./charts/db_{region}_{y}_{m}_{d}_2.png')
document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/db_{region}_{y}_{m}_{d}_3.png')   

In [40]:
# Document
document.add_page_break()

In [41]:
document.save('demo.docx')

## 1.6 자산 관리 - K8s

In [ ]:
base_path = r"./data/1_6_202206_자산관리_k8s_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df)

In [ ]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(df_filtered, month)
    chart.save(f'./charts/k8s_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/k8s_{region}_{y}_{m}_{d}_2.png')

In [ ]:
# Document
document.add_paragraph('자산 관리(k8s)', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/k8s_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/k8s_{region}_{y}_{m}_{d}_2.png')   

In [ ]:
# Document
# addSummaryTable(df_filtered, regex, month)
document.add_page_break()

## 1.7 용량 관리

In [ ]:
base_path = r"./data/1_7_202206_용량관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df)

In [ ]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(df_filtered, month)
    chart.save(f'./charts/cpm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/cpm_{region}_{y}_{m}_{d}_2.png')

In [ ]:
# Document
document.add_paragraph('용량(이슈) 관리', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/cpm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/cpm_{region}_{y}_{m}_{d}_2.png')   

In [ ]:
# Document
addSummaryTable(df_filtered, regex, month)
document.add_page_break()

In [ ]:
document.add_paragraph(f'자원 현황표', style='List Bullet')

## 1.8 백업 관리

In [ ]:
base_path = r"./data/1_8_202206_백업관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df)

In [ ]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(df_filtered, month)
    chart.save(f'./charts/bm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/bm_{region}_{y}_{m}_{d}_2.png')

In [ ]:
# Document
document.add_paragraph('백업 관리', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/bm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/bm_{region}_{y}_{m}_{d}_2.png')   

In [ ]:
# Document
# addSummaryTable(df_filtered, regex, month)
document.add_page_break()

## 1.9 보안 관리

In [ ]:
base_path = r"./data/1_9_202206_보안관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df)

In [ ]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(df_filtered, month)
    chart.save(f'./charts/sm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/sm_{region}_{y}_{m}_{d}_2.png')

In [ ]:
# Document
document.add_paragraph('보안 관리', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/sm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/sm_{region}_{y}_{m}_{d}_2.png')   

In [ ]:
# Document
addSummaryTable(df_filtered, regex, month)
document.add_page_break()

## 1.10 하드웨어 정기점검

In [ ]:
base_path = r"./data/1_10_202206_정기점검_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df)

In [ ]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(df_filtered, month)
    chart.save(f'./charts/rc_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/rc_{region}_{y}_{m}_{d}_2.png')

In [ ]:
# Document
document.add_paragraph('정기점검', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/rc_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/rc_{region}_{y}_{m}_{d}_2.png')   

In [ ]:
# Document
# addSummaryTable(df_filtered, regex, month)
document.add_page_break()

In [ ]:
import datetime
from chart import *

# Data Loading
base_path = r"./data/1_7_202206_용량관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)
df

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# Pivoting - [df['월']=='6월') & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
pivot, total = getPivotTable(df)
print(total)
pivot

In [ ]:
# ETL
total = Data.count()
data = getdata()
chart = getTable(data)

### 비중 :: Piot Table

In [ ]:
pivot = getPivotTable(df)

In [ ]:
source = pivot
chart = get_pieChart(source)
chart

In [ ]:
chart.save(f'./charts/mon_{y}_{m}_{d}_1.png')

In [ ]:
document.add_paragraph('1.1 서비스 관리', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### 시계열 :: LineChart

In [ ]:
pTable

In [ ]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/2.png')
document.add_picture('charts/2.png')

In [ ]:
df_filtered[df_filtered['월'] == '6월'].sort_values(['리전']).reset_index()

### 시계열 :: BarChart

In [ ]:
source=pTable
chart = get_barChart(source)
chart

In [ ]:
chart.save('charts/3.png')
document.add_picture('charts/3.png')

### 시계열 - LineChart or BarChart

### ALL

In [ ]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/4.png')
document.add_picture('charts/4.png')

In [ ]:
source

In [ ]:
# Write to Word
document.add_paragraph('1.1 모니터링', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### by Region

In [ ]:
regions = df['리전'].unique()

for region in regions:
    print(region)

#### KR

In [ ]:
regex = '^(KR|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/5.png')
document.add_picture('charts/5.png')

#### CN

In [ ]:
regex = '^(CN|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/6.png')
document.add_picture('charts/6.png')

#### NA

In [ ]:
regex = '^(NA|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/7.png')
document.add_picture('charts/7.png')

#### EU

In [ ]:
regex = '^(EU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/8.png')
document.add_picture('charts/8.png')

#### SG

In [ ]:
regex = '^(SG|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/9.png')
document.add_picture('charts/9.png')

#### RU

In [ ]:
regex = '^(RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/10.png')
document.add_picture('charts/10.png')

In [ ]:
document.save('word.docx')

In [ ]:
import datetime
from chart import *

# Data Loading
base_path = r"./data/1_8_202206_백업관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# Pivoting - [df['월']=='6월') & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
pivot, total = getPivotTable(df)
print(total)
pivot

In [ ]:
# ETL
total = Data.count()
data = getdata()
chart = getTable(data)

### 비중 :: Piot Table

In [ ]:
pivot = getPivotTable(df)

In [ ]:
source = pivot
chart = get_pieChart(source)
chart

In [ ]:
chart.save(f'./charts/mon_{y}_{m}_{d}_1.png')

In [ ]:
document.add_paragraph('1.1 서비스 관리', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### 시계열 :: LineChart

In [ ]:
pTable

In [ ]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/2.png')
document.add_picture('charts/2.png')

In [ ]:
df_filtered[df_filtered['월'] == '6월'].sort_values(['리전']).reset_index()

### 시계열 :: BarChart

In [ ]:
source=pTable
chart = get_barChart(source)
chart

In [ ]:
chart.save('charts/3.png')
document.add_picture('charts/3.png')

### 시계열 - LineChart or BarChart

### ALL

In [ ]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/4.png')
document.add_picture('charts/4.png')

In [ ]:
source

In [ ]:
# Write to Word
document.add_paragraph('1.1 모니터링', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### by Region

In [ ]:
regions = df['리전'].unique()

for region in regions:
    print(region)

#### KR

In [ ]:
regex = '^(KR|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/5.png')
document.add_picture('charts/5.png')

#### CN

In [ ]:
regex = '^(CN|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/6.png')
document.add_picture('charts/6.png')

#### NA

In [ ]:
regex = '^(NA|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/7.png')
document.add_picture('charts/7.png')

#### EU

In [ ]:
regex = '^(EU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/8.png')
document.add_picture('charts/8.png')

#### SG

In [ ]:
regex = '^(SG|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/9.png')
document.add_picture('charts/9.png')

#### RU

In [ ]:
regex = '^(RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/10.png')
document.add_picture('charts/10.png')

In [ ]:
document.save('word.docx')

In [ ]:
# Data Loading
base_path = r"./data/1_9_202206_보안관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.info()

In [ ]:
df.info()

In [ ]:
# Pivoting - [df['월']=='6월') & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
pivot, total = getPivotTable(df)
print(total)
pivot

In [ ]:
# ETL
total = Data.count()
data = getdata()
chart = getTable(data)

### 비중 :: Piot Table

In [ ]:
pivot = getPivotTable(df)

In [ ]:
source = pivot
chart = get_pieChart(source)
chart

In [ ]:
chart.save(f'./charts/mon_{y}_{m}_{d}_1.png')

In [ ]:
document.add_paragraph('1.1 서비스 관리', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### 시계열 :: LineChart

In [ ]:
pTable

In [ ]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/2.png')
document.add_picture('charts/2.png')

In [ ]:
df_filtered[df_filtered['월'] == '6월'].sort_values(['리전']).reset_index()

### 시계열 :: BarChart

In [ ]:
source=pTable
chart = get_barChart(source)
chart

In [ ]:
chart.save('charts/3.png')
document.add_picture('charts/3.png')

### 시계열 - LineChart or BarChart

### ALL

In [ ]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/4.png')
document.add_picture('charts/4.png')

In [ ]:
source

In [ ]:
# Write to Word
document.add_paragraph('1.1 모니터링', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### by Region

In [ ]:
regions = df['리전'].unique()

for region in regions:
    print(region)

#### KR

In [ ]:
regex = '^(KR|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/5.png')
document.add_picture('charts/5.png')

#### CN

In [ ]:
regex = '^(CN|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/6.png')
document.add_picture('charts/6.png')

#### NA

In [ ]:
regex = '^(NA|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/7.png')
document.add_picture('charts/7.png')

#### EU

In [ ]:
regex = '^(EU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/8.png')
document.add_picture('charts/8.png')

#### SG

In [ ]:
regex = '^(SG|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/9.png')
document.add_picture('charts/9.png')

#### RU

In [ ]:
regex = '^(RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/10.png')
document.add_picture('charts/10.png')

In [ ]:
document.save('word.docx')

In [ ]:
# Data Loading
base_path = r"./data/1_10_202206_정기점검_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
# Pivoting - [df['월']=='6월') & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
pivot, total = getPivotTable(df)
print(total)
pivot

In [ ]:
# ETL
total = Data.count()
data = getdata()
chart = getTable(data)

### 비중 :: Piot Table

In [ ]:
pivot = getPivotTable(df)

In [ ]:
source = pivot
chart = get_pieChart(source)
chart

In [ ]:
chart.save(f'./charts/mon_{y}_{m}_{d}_1.png')

In [ ]:
document.add_paragraph('1.1 서비스 관리', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### 시계열 :: LineChart

In [ ]:
pTable

In [ ]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/2.png')
document.add_picture('charts/2.png')

In [ ]:
df_filtered[df_filtered['월'] == '6월'].sort_values(['리전']).reset_index()

### 시계열 :: BarChart

In [ ]:
source=pTable
chart = get_barChart(source)
chart

In [ ]:
chart.save('charts/3.png')
document.add_picture('charts/3.png')

### 시계열 - LineChart or BarChart

### ALL

In [ ]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/4.png')
document.add_picture('charts/4.png')

In [ ]:
source

In [ ]:
# Write to Word
document.add_paragraph('1.1 모니터링', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### by Region

In [ ]:
regions = df['리전'].unique()

for region in regions:
    print(region)

#### KR

In [ ]:
regex = '^(KR|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/5.png')
document.add_picture('charts/5.png')

#### CN

In [ ]:
regex = '^(CN|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/6.png')
document.add_picture('charts/6.png')

#### NA

In [ ]:
regex = '^(NA|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/7.png')
document.add_picture('charts/7.png')

#### EU

In [ ]:
regex = '^(EU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/8.png')
document.add_picture('charts/8.png')

#### SG

In [ ]:
regex = '^(SG|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/9.png')
document.add_picture('charts/9.png')

#### RU

In [ ]:
regex = '^(RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/10.png')
document.add_picture('charts/10.png')

In [ ]:
document.save('word.docx')

## Save Document

In [ ]:
document.save(f'{month}_{region}_월간보고서_{y}_{m}_{d}.docx')